## Holder Table Function

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/Globtim.jl`


In [2]:
# Constants and Parameters
const n, a, b = 2, 10, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = HolderTable # Objective function

HolderTable (generic function with 1 method)

In [8]:
d = 18 # Initial Degree 
SMPL = 40 # Number of samples
center = [0.0, 0.0]
TR = test_input(n, center, (alpha, delta), tol_l2, (0.0, 0.0), scale_factor, 1.0, f)
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

current L2-norm: 0.03248275406196476
Number of samples: 40
current L2-norm: 0.02243641155105562
Number of samples: 40


In [4]:
@polyvar(x[1:n]) # Define polynomial ring 

(Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}[x₁, x₂],)

In [9]:
real_pts_cheb = solve_polynomial_system(x, n, d, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
real_pts_lege = solve_polynomial_system(x, n, d, pol_lege.coeffs; basis=:legendre, bigint=true)
# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, scale_factor)
df_lege = process_critical_points(real_pts_lege, f, scale_factor)

Tracking 289 paths...   1%|▎                            |  ETA: 0:11:41



Tracking 289 paths...  27%|███████▉                     |  ETA: 0:00:14



Tracking 289 paths...  55%|████████████████             |  ETA: 0:00:04



Tracking 289 paths...  82%|███████████████████████▋     |  ETA: 0:00:01



Tracking 289 paths... 100%|█████████████████████████████| Time: 0:00:05
  # paths tracked:                  289
  # non-singular solutions (real):  289 (181)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         289 (181)
Tracking 289 paths...   1%|▎                            |  ETA: 0:13:52



Tracking 289 paths...   1%|▍                            |  ETA: 0:07:19



Tracking 289 paths...   2%|▌                            |  ETA: 0:06:19



Tracking 289 paths...   2%|▋                            |  ETA: 0:05:25



Tracking 289 paths...   3%|▊                            |  ETA: 0:04:15



Tracking 289 paths...   3%|█                            |  ETA: 0:03:30



Trac

Row,x,y,z
,Float64,Float64,Float64
1,9.56062,9.95535,-3.4775
2,6.36225,9.74765,-1.12027
3,3.08136,9.8346,-0.540102
4,2.71112e-13,9.82686,-2.0952e-12
5,-3.08136,9.8346,-0.540102
6,-6.36225,9.74765,-1.12027
7,-8.10344,9.91092,-18.5493
8,-4.8841,9.84946,-10.9313
9,-1.55031,9.82499,-8.033


In [10]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_lege.grid
z_coords = pol_lege.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Holder Table Function",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Scale the domain
    scale_factor = pol_cheb.scale_factor  # Using the scale factor from your polynomial

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot with scaled domain
        surface!(ax, scale_factor .* x_unique, scale_factor .* y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points with scaled domain
        scatter!(ax, scale_factor .* coords[sample_indices, 1],
            scale_factor .* coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets with scaled domain
        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=2,
            color=:black,
            label="Sampled Data Core")

        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=4,
            color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points with scaled domain
    scatter!(ax, scale_factor .* df_cheb.x,
        scale_factor .* df_cheb.y,
        df_cheb.z,
        markersize=10,
        color=:orange,
        label="Chebyshev approximant critical points")

    scatter!(ax, scale_factor .* df_lege.x,
        scale_factor .* df_lege.y,
        df_lege.z,
        markersize=10,
        color=:yellow,
        label="Legendre approximant critical points")

    display(fig)
end

GLMakie.Screen(...)